In [1]:
import chromadb
import numpy as np
import pandas as pd

# Vector DB - Chroma DB 구축

In [2]:
df = pd.read_csv('날짜별_의정활동.csv')

In [7]:
client = chromadb.PersistentClient(path="chromadb")

collection = client.get_or_create_collection(
    name="law_table",
    metadata={"hnsw:space": "cosine"}
)

In [8]:
# 인덱스
ids = []
# 메타 데이터
doc_meta = []
# 벡터로 변환 저장할 텍스트 데이터로 ChromaDB에 Embedding 데이터가 없으면 자동으로 벡터로 변환해서 저장한다고 한다. 
documents = []

In [9]:
for idx in range(len(df)):
    # iloc: index 위치를 기반으로 df에서 행을 선택
    item = df.iloc[idx]
    id = item['의안명'].replace(' ', '-')
    document = f"(의안명: {item['의안명']}) (의안 일자: {item['일자']}) (의안번호: {item['의안번호']}) - 단계:{item['단계']}, 세부단계: {item['세부단계']}, 소관위원회: {item['소관위원회']}"
    meta = {
        "카테고리": item['카테고리']
    }
    ids.append(id)
    doc_meta.append(meta)
    documents.append(document)

# DB 저장

In [10]:
collection.add(
    documents = documents,
    metadatas=doc_meta,
    ids=ids
)

# DB 쿼리

In [14]:
collection.query(
    query_texts=["재산"],
    n_results=5,
    where={"카테고리":"경제"},
    where_document={"$contains": "재산"}
)

{'ids': [['공유재산특례제한법안(정부)', '공유재산-및-물품-관리법-일부개정법률안(이용호의원-등-11인)']],
 'distances': [[0.48423445224761963, 0.7289931774139404]],
 'metadatas': [[{'카테고리': '경제'}, {'카테고리': '경제'}]],
 'embeddings': None,
 'documents': [['(의안명: 공유재산특례제한법안(정부)) (의안 일자: 2023-11-22) (의안번호: 2113019) - 단계:위원회 심사, 세부단계: 회의, 소관위원회: 행정안전위원회',
   '(의안명: 공유재산 및 물품 관리법 일부개정법률안(이용호의원 등 11인)) (의안 일자: 2023-11-22) (의안번호: 2125548) - 단계:위원회 심사, 세부단계: 회부, 소관위원회: 행정안전위원회']]}